# Ultrafast sparse binning clustering enhanced by ML track scoring

###  Yuval Reina, Tel-Aviv Israel, [Yuval.reina@gmail.com](Yuval.reina@gmail.com)
###  Trian Xylouris, Frankfurt am Main Germany, [t.xylouris@gmail.com](t.xylouris@gmail.com)

August 2018


#### Competition Name: TrackML
#### Team Name: Yuval & Trian
#### Private Leaderboard Score: 0.80414
#### Private Leaderboard Place: 7

## Summery

This method is using sparse binning to perform ultrafast clustering, tracks are first chosen according to their length, and later are scored and merged using a Machine Learning algorithm. At the final stage the tracks are expanded by adding the closest hits to the track.

**The biggest advantage of using clustering by sparse binning is speed**. This method can score 0.5 in just 40 sec using python on a single core, and it could be much faster if it was written in C++ and even faster by using paralleling on CPU or GPU. **The computational complicity of the clustering part in the algorithm is O(N)**. The feature calculation and binning can be done for every hit independent of the other hits and all hits are needed only for counting the number of hits in every bin. (The Python implementation uses np.unique which is actually O(NlogN)  ). We believe that with a careful implementation in C++  a score of 0.5 can be achieved in less than 10 sec which makes this algorithm an essential step in every fast algorithm (we will explain this statement later).

The second stage in the algorithm in ML merging. The fastest way to merge 2 solutions is by letting each hit choose the track with the highest number of hits, this method is used in the clustering main loop. This method is limited because the number of hits is not always a good indication for a good track. The ML algorithm is uses many features which describe the track and is able to distinguish between good and bad tracks. Unlike other ML solution presented in Kaggle, our ML algorithm does not check the helix itself, as the clustering part already took care of this.

The last part in the algorithm is quite straightforward, track expanding is done by choosing the long tracks and adding to these tracks the closest loose hits (i.e. hits from short tracks). 

The full solution is then: 
•	Run clustering a few times
•	Use ML to merge
•	Expand 

In the solution we submitted we used about 7 runs of clustering, using 100000 loops each, merged and expanded twice (The 2nd time was just to add about 0.003 to the score by using a loophole in the score definition [(1)](https://www.kaggle.com/c/trackml-particle-identification/discussion/60638#354053)) 


## Chapter 1: Methods and Results

## Clustering using Sparse Binning

The basic idea of clustering it to relate to every hit a set of numbers (features) and then to group hits with close feature together.
The first stage in clustering is finding features which would describe a track and which we can calculate separately for every hit. 
We have published a post on criteria for good features [(2)](https://www.kaggle.com/c/trackml-particle-identification/discussion/61590).

### Features 
Due to the magnetic field in the experiment, the particles form helixes, which travel along the z-axis. Each hit in the x-y-z space can be the member of any particular helix in a certain family of helixes. The idea of the algorithm is to go over each possible member of this family of helixes, and look at how many of all hits may belong to this particular member. The hits are then assigned to the helix (=track) with the most possible hits.

We start by assuming the particle is formed in a small cylinder around the origin i.e. the approximate starting point of the helix is '(0,0,z0)'. (According to the introduction papers '|z0|<5.5mm').

This kind of helix can be defined by 3 numbers, its Radius, its tangential angle at the origin in the xy plan (=the direction of the particle when it is created, in xy plan), the slop of the helix (how fast the particle moves in the Z direction compared to its velocity in xy).

We will define:

R – helix radius

$\theta$ – tangential angle in the xy plan

$\phi$ – Slope

$$ \begin{align*}
&\theta = \arctan\frac{py}{px} \label{x1} \\
&\phi = \arctan\frac{pz}{\sqrt{px^2+py^2}} \label{x2}
\end{align*}
$$

where px, py, pz, are the particle’s initial momentum 

If we take a family of helixes with a radius R, we can calculate for every hit '(x,y,z)' the values for theta and phi.
We define

$$ \begin{align*}
&kt = \frac{1}{2R} \label{x3} \\
\end{align*}
$$

We use kt>0 for particle rotating to the CW and kt<0 for CCW rotation

Using sum trigonometry, we will get:

$$ \begin{align*}
&\theta\_ = \arctan\frac{py}{px}\label{x4} \\
\\
&rr=\sqrt{x^2+y^2} \label{x5} \\
\\
&\Delta\theta=\arcsin{(kt\cdot rr)} \label{x6} \\
\\
&\theta=\theta\_+\Delta\theta \label{x7}\\
\end{align*}
$$
  
(At this stage we ignore particles that rotate more then $\pi$ radians)
$$ \begin{align*}
&\phi\_ = \arctan\frac{(z-z0)\cdot kt}{\Delta\theta} \label{x8}
\end{align*}
$$

$\theta$ and $\phi\_$ will be our features, but we need to tweak them a little to become useful.

If $\theta_1 = \phi+\epsilon$ and $\theta_1 = \phi-\epsilon$,  $\theta_1 – \theta_2= 2\epsilon$ on a circle, but if we calculate $\theta_1 – \theta_2$ we will get $2\pi+2\epsilon$. To solve this issue we will use 2 features instead:

$$ \begin{align*}
&sint = \sin(\theta) \label{x9}\\
&cost = \cos(\theta) \label{x10}
\end{align*}
$$

$\phi\_$ on the other hand is in the range $[-\pi/2,\pi/2]$, and does not have the above issue, but $\phi\_$ distribution is far from being uniform (for an extensive discussion about its distribution look here [(3)](https://www.kaggle.com/c/trackml-particle-identification/discussion/63250#latest-371940)). 

The solution we found suitable is by defining:
$$ \begin{align*}
&\phi = \arctan\frac{(z-z0)\cdot kt}{3.3\cdot\Delta\theta}\frac{2}{\pi} \label{x11}
\end{align*}
$$

(the last edition is for scaling to a [-1,1] range)  


### Sparse Binning Clustering
Binning is the easiest way to cluster, in this method we divide the values of each feature to constant areas and a cluster is formed by all the hits whos features are all in the same area. 

As an example, if we have one feature in the range `[-1,1]` we can decide to divide it to 20 equal bins of size `0.1: [-1,-0.9), [-0.9,0.8), …,[0.9,1]`. We will assign every bin a number 0 , …, 19 and define the hit’s feature with this value. If we have 3 features for a hit we will define that hit with the 3 values of bins. As an example:

`
Hit #1 – (12,15,0)
Hit #2 – (11,5,2)`

Two hits which have the exact 3 values, are clustered together to form track candidate.

In general, the bins don’t have to spread uniformly, and every feature can have a different number of bins.

In Sparse Binning we use a very simple binning algorithm. We start by using uniform spread bins which can be easily calculated:

For feature `F1` in the range `[-1,1]` we define `B1 = int(F1*k1)` where the number of bins is `2*k1`

i.e. we multiply the feature by half the number of bins and take the integer part.

To combine few features together and get the cluster id (track_id) we just offset each feature by multiplying it by a number which is big enough. 

If we have 2 features F1, F2 and we want to have 2*k1, 2*k2 bins for each feature, we can define:

` (a) Track_id = int(F1*k1) + (2*k1+1)*int(F2*k2)`

And that’s it clustering is finished!!

This calculation takes about 4mSec for 120,000 hits on Kaggle kernel platform compared to about 950mSec for dbscan which is the most popular clustering algorithm used by the competitors in the TrackML Challenge. 

This type of clustering moves the bottleneck of the full solution, form clustering to merging (will be discussed in the next section).

One weakness of the binning algorithm is its sensitivity caused, among other things, by the hard boarders between bins. As an example, let’s take two hits:

`
Hit1 – with features (-1e-7,0.1,0.3)
Hit2 – with features (1e-7,0.1,0.3)`

If we use the calculation above (1), Hit1, Hit2 will never cluster together no matter how many bins we’ll have.

This issue is solved by adding a random number in the range `[0,1)` to every feature in the binning calculation, making (a):

`(b) Track_id = int(F1*k1+r1) + (2*k1+1)*int(F2*k2+r2)`

Where r1, r2 are random numbers with uniform distribution in the range `[0,1)`. r1, r2 are changed every time we cluster.

When compared to dbscan, sparse clustering has one big disadvantage, it’s sensitivity, it can miss some of the hits because they fall in another bin. This sensitivity is also its strength, as it doesn’t cluster many wrong hits (outliners) which degrade the maximum achievable score. 


### Simple Merging
Every time we cluster with a set of kt, z0, every hit gets a new track_id, to have a full solution we want to merge two lists of track id’s together. The most efficient way we found was to measure the length of each track in the two lists, and let every hit choose the longest track it can.

If Hit #1 has track_id=1 in list #100 and track_id=10000002 in list #101 (the track_id’s in both lists should be completely different) and in list #100 there are 10 hits with track_id=1 while in list #101 there are 7 hits with track_id=10000002, hit#1 will choose track_id=1.

To do this, we need to measure the track’s length. The fastest way we know to do this in Python is by using numpy.unique. This operation takes 11mSec for 120000 hits on Kaggle kernel platform. The numpy.unique computational complexity is O(NlogN) because it sorts the values. While if we have enough memory, this task can easily be done in O(N). (We can also reduce the amount of memory needed by hashing). 

In our main loop we measure the length of a track twice, the first time we do as we described above to create a track candidate and then we measure again to make sure the track is still long enough as hits may decide to choose another track in the first round. 

The double track length measurement makes this the bottleneck of the algorithm


### Clustering – the full algorithm
The algorithm iterate over random selected pairs – (z0,kt). For every pair the hits’ features are calculated, and clustering is performed by the sparse binning algorithm. The new clustering result is then merged with previous ones with the merging algorithm described above. 

Every 500 loops an extra step is taken. Tracks are examined for hits coming from the same detector (equal Volume_id, Layer_id, and Module_id) or from the same layer. There can’t be two hits from the same detector or 3 from the same layer. The extra hits, are discovered by measuring the distance between the hit and the center of gravity of the track. These hits are removed from the track. 

In this step the algorithm also permanently set tracks which are long enough (hits won’t we taken away or added to this track any more).

While running the algorithm can change the number of bins per feature and the length of the minimal track to set, these settings are user defined parameters.


### Clustering results:
The score common score after the clustering algorithm with as a function of  the number of (z0,kt) pairs used is presented in the following table (measured on event 000001000 from the training set – usually 0.015 below final LB score):

|   Pairs   |  Score |
|-----------|--------|
|    1000	|  0.51  |
|    1600	|  0.56  |
|    5500	|  0.636 |
|  100000	|  0.73  |

The score plateau after about 90000 pairs and wouldn’t get higher


###  Machine Learning
We run the binning algorithm 8 times in total, and merge the resulting solutions, using a machine algorithm model. In particular, we use LightGBM, but expect that a deep neural network or even Logistic Regression would yield similar results. The machine learning algorithm predicts the probabilitiy that a track is good, based on what it has learned from the training events. In particular, it looks at 13 features for each track. This improvement increases the score from ~0.73 to ~0.76. 

Note: if we increased the number of merged solutions to 30, we expect to get ~0.78 instead of ~0.76.

###  Track Extension
The major drawback of the binning clustering algorithm is its sensitivity. The bins sharp boarders can leave a hit out of the track although it is very close to the other hits. To overcome this issue, a track expanding algorithm is the final step in the full solution.

The final clustering and merging solution it taken as a basis for this step.

First, the algorithm tries to improve the (z0,kt) pair for each track. It does this by searching for a pair which will minimize the standard deviation of the track hits.

 Using these refined (kt,z0) values. The hits which are closest to track are added to it. The distance between a hit and a track is measured by the difference between the its calculated features and the mean of the tracks features.
 
We also tested a variant of this algorithm which measure the minimal distance to one of the track’s hit (as in nearest neighbor algorithm), this variant performed slightly better but was much slower.  

The improvement gained by expanding depends on the score after the previous stages.

0.63 score can jump to 0.73,

0.73 would go to 0.79

and in our final run 0.76 was improved to 0.804

This input to this stage must come from good clustering algorithm and cascaded extending will usually degrade the score. 


### Outlook

1.	As a full solution our approach can be easily optimized for speed:
    -	The clustering algorithm can be improved as described above. Farther improvement for real world application can be achieved by massively paralleling the feature calculation using a GPU (as we explained above, every hit is treated separately, which makes the algorithm very suitable for parallel processing).
    -	Both the ML and the expanding algorithm where not optimized at all as they weren’t the bottleneck of the solution. As an example, the expanding algorithm recalculates the features for all the loose hits every time. Calculating the features only for hits with z>0 while expanding tracks with z>0 will immediately cut the running time of the algorithm by half. 
2.	Elements of our approach may be beneficial for any final solution to this problem.
   - In particular, our binning algorithm creates a lot of good candidate tracks in a short period of time. It may be helpful to employ this at the beginning of a solution, in order to very quickly (<1 minute) detect 50% of all tracks (use small bins). One can then continue with running a different algorithm on the remaining hits.
   - Similarly, the machine learning algorithm, as well as the employed parameters and track extension algorithm may improve any final result, while just adding minutes (or less, after optimization) to the total runtime.
3.	In our algorithm we didn’t do any adjustments for uneven magnetic field. If we incorporate @CPMP’s findings [here (3)](https://www.kaggle.com/c/trackml-particle-identification/discussion/63250#latest-372564). The score of the algorithm improves immediately by 0.02 to about 0.82. 
4.	An obvious short-coming of   main approach is, that it is not suited to find tracks, which originate far from the origin. We did try to adjust our algorithm to work also in that situation, but, had minimal success so far.




### References 

(1)  @Grzegorz Sionkowski comment in *"how to score a track is good or not"* [https://www.kaggle.com/c/trackml-particle-identification/discussion/60638#354053](https://www.kaggle.com/c/trackml-particle-identification/discussion/60638#354053)

(2) *"Criteria for good features"* [(https://www.kaggle.com/c/trackml-particle-identification/discussion/61590)](https://www.kaggle.com/c/trackml-particle-identification/discussion/61590)

(3) @CPMP *"Solution #9"* [https://www.kaggle.com/c/trackml-particle-identification/discussion/63250#latest-372564](https://www.kaggle.com/c/trackml-particle-identification/discussion/63250#latest-372564)

## Chapter 2: Running full pipeline for train event 1000

### Introduction

We will demonstrate how to get a score of TBD for train event 1000, in TBD minutes, by using our aforementioned method. The steps are: 
- Create 2 initial solutions: 2x use 5.500 pairs of (kt,z0) for binning (in our original solution, we use 100.000 pairs)
- Merge the 2 solutions using a machine learning algorithm (in our original solution, we use 8)
- Extend the tracks (in our original solution, we run this twice, while at the first time, we extend hits according to whether their values for (1) and (2) are close enough to at least 1 hit, instead of the mean of the track)

Import necessary packages and load train event 1000, which will be the event we will be working on:

In [61]:
from IPython.display import HTML
import numpy as np
import sys
sys.path.insert(0, 'other/')
import pandas as pd
import datetime
import os
from ipywidgets import FloatProgress,FloatText
from IPython.display import display
import time
import pdb
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import gc
import cProfile
from tqdm import tqdm_notebook
%matplotlib inline
#make wider graphs
sns.set(rc={'figure.figsize':(12,5)})
plt.figure(figsize=(12,5))
path='files/'

from functions.other import calc_features, get_event, score_event_fast, load_obj
from functions.expand import *
from functions.cluster import *
from functions.ml_model import merge_with_probabilities,precision_and_recall,get_features,get_predictions

# the following two lines are for changing imported functions, and not needing to restart kernel to use their updated version
%load_ext autoreload
%autoreload 2

event_num=0
event_prefix = 'event00000100{}'.format(event_num)
hits, cells, particles, truth = get_event(path,event_prefix)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<Figure size 864x360 with 0 Axes>

### Clustering
Define parameters and run clustering, twice:

In [57]:
history=[]
weights={'pi':1,'theta':0.15}
stds={'z0':7.5, 'kt':7.5e-4}
d =    {'sint':[225,110,110,110,110,110],
        'cost':[225,110,110,110,110,110],
          'phi':[550,260,260,260,260,260],
        'min_group':[11,11,10,9,8,7],
        'npoints':[200,200,200,200,100,100]}  # quick run for testing
        #'npoints':[500,2000,1000,1000,500,500]}
filters=pd.DataFrame(d)
#nu=500
nu=100

resa1=clustering(hits,stds,filters,phik=3.3,nu=nu,truth=truth,history=history)
resa1["event_id"]=event_num
score = score_event_fast(truth, resa1.rename(index=str, columns={"label": "track_id"}))
print("Your score: ", score)

resa2=clustering(hits,stds,filters,phik=3.3,nu=nu,truth=truth,history=history)
resa2["event_id"]=event_num
score = score_event_fast(truth, resa2.rename(index=str, columns={"label": "track_id"}))
print("Your score: ", score)

FloatText(value=0.0, description='full score:')

FloatText(value=0.0, description='score:')

FloatText(value=0.0, description='s rate:')

FloatText(value=0.0, description='add score:')

FloatText(value=120939.0, description='Rest size:')

...

FloatText(value=120939.0, description='Rest size:')

FloatText(value=120939.0, description='Group size:')

FloatText(value=0.0, description='filter:')

100%|██████████| 1000/1000 [00:37<00:00, 26.32it/s]

took 38.06680 sec
Your score:  0.49943215791999995


## Employ Machine Learning
### General strategy
- Produce different submission candidates sub_1, sub_2, ..., sub_N
- Create a machine learning model, which gives probabilities between 0 and 1 for each track candidate
- Merge two submission candidates by assigning to each hit the track, which has higher probability
- Merge the submission candidates successively (sub_1 and sub_2 to sub_12, sub_12 and sub_3 to sub_123, etc.) to get the final submission

[Note: For our final solution, the methods described in this chapter gave around +0.01 to the LB score. Certain benefits from these methods have already been captured by the function, which expands the tracks.]

### Create machine learning model (LightGBM)
- Training data: Use the truth file of the train events 3-5 to get true tracks (target=1). To get wrong tracks (target=0) use generated clustering submissions on those latter events. In particular, we consider each track candidate, for which not all hits belong to the same particle_id, as a wrong track. Also, choose only tracks which have at least length 4, to slightly optimize compute time at negligible cost.
- Use 13 features per track: 
 - variance of x,y,z (these are the most important)
 - minimum of x,y,z
 - maximum of x,y,z
 - mean of z
 - volume_id of first hit 
 - number of clusters per track (i.e. are there many hits, which are close together)
 - number of hits / number of clusters   
- Validation data: Same as with training data, but use the 3 training events 0,1,2

[Note: We also tried many more features (compare below dataframe df_train), but they gave only small additional gains, so we just kept those 13 features for simplicity. In fact, many other interesting features, such as number of hits, number of different volumes crossed etc are closely related to the above used ones.]

We have prepared the training and test data and load it directly from a pkl-file:

In [46]:
df_train=load_obj('files/df_train_v2-reduced.pkl')
df_test=load_obj('files/df_test_v1.pkl')
y_train=df_train.target.values
y_test=df_test.target.values
print("The dataframe with all features:")
display(df_train.head())
print("Features for each track:",df_train.columns.values)

The dataframe with all features:


,cells_length,cells_max,cells_mean,cells_min,cells_var,elayer,emodule,evolume,ex,ey,...,ymean,ymedian,ymin,yvar,zmax,zmean,zmedian,zmin,zvar,event_id
0,4.0,0.341868,0.119788,2.044216e-02,0.017071,6.0,29.0,14.0,-290.533997,-463.497009,...,-256.535034,-286.803009,-463.497009,26005.789062,1801.500000,951.502136,1031.050049,107.703003,3.882222e+05,3
1,8.0,0.450497,0.071357,2.354972e-14,0.020859,2.0,145.0,12.0,-231.250000,351.147003,...,143.303238,92.862198,20.085199,10862.027344,-244.580994,-1334.507324,-958.000000,-2951.500000,7.111888e+05,3
2,11.0,0.680366,0.167157,1.636995e-02,0.035665,6.0,66.0,18.0,470.838989,768.648010,...,378.134766,309.616516,30.304199,72788.718750,1802.500000,782.384949,561.099976,64.440201,3.828446e+05,3
3,3.0,1.142238,0.461141,1.039750e-01,0.232131,12.0,26.0,18.0,94.234398,-865.289001,...,-450.889038,-434.042480,-865.289001,130411.406250,2947.500000,1532.954224,1469.185059,245.947006,1.497747e+06,3
4,10.0,0.791248,0.182531,3.750272e-02,0.052045,6.0,27.0,16.0,135.772003,-984.747986,...,-389.764557,-330.183014,-984.747986,91312.648438,-71.102097,-869.824463,-752.000000,-2152.500000,4.269275e+05,3


Features for each track: ['cells_length' 'cells_max' 'cells_mean' 'cells_min' 'cells_var' 'elayer'
 'emodule' 'evolume' 'ex' 'ey' 'ez' 'hitids' 'nclusters' 'nhits'
 'nhitspercluster' 'nsensors' 'nvolumes' 'particle_id' 'sentence'
 'sentence_count' 'sentence_value' 'slayer' 'smodule' 'svolume' 'sx' 'sy'
 'sz' 'target' 'xmax' 'xmean' 'xmedian' 'xmin' 'xvar' 'ymax' 'ymean'
 'ymedian' 'ymin' 'yvar' 'zmax' 'zmean' 'zmedian' 'zmin' 'zvar' 'event_id']


In the competition, we used roughly 250 events for training, but the additional improvement to just using 13 events is not too big. We now create the LightGBM model, using the mentioned training data and features:

In [47]:
import lightgbm
s=time.time()
# choose which features of the tracks we want to use:
columns=['svolume','nclusters', 'nhitspercluster', 'xmax','ymax','zmax', 'xmin','ymin','zmin', 'zmean', 'xvar','yvar','zvar']
rounds=1000
round_early_stop=50
parameters = { 'subsample_for_bin':800, 'max_bin': 512, 'num_threads':8, 
               'application': 'binary','objective': 'binary','metric': 'auc','boosting': 'gbdt',
               'num_leaves': 128,'feature_fraction': 0.7,'learning_rate': 0.05,'verbose': 0}
train_data = lightgbm.Dataset(df_train[columns].values, label=y_train)
test_data = lightgbm.Dataset(df_test[columns].values, label=y_test)
model = lightgbm.train(parameters,train_data,valid_sets=test_data,num_boost_round=rounds,early_stopping_rounds=round_early_stop,verbose_eval=50)
print('took',time.time()-s,'seconds')

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.954964
[100]	valid_0's auc: 0.960705
[150]	valid_0's auc: 0.962809
[200]	valid_0's auc: 0.963605
[250]	valid_0's auc: 0.964108
[300]	valid_0's auc: 0.964196
[350]	valid_0's auc: 0.964214
Early stopping, best iteration is:
[317]	valid_0's auc: 0.964268
took 1.4954051971435547 seconds


### Judge machine learning model

We doublecheck the model's performance, by calculating its precision, recall and accuracy on the validation set:

[Note: For ML to be helpful in our situation, it needs to distinguish correct from wrong tracks with very high precision=true_positives/(false_positives+true_positives)). Also, it needs to do so for various sets of track candidates (especially such, which are generated if one tries to find tracks which originate far away from the origin; in those situations, often a lot of bad candidates are produced).]

In [50]:
y_test_pred=model.predict(df_test[columns].values)
precision, recall, accuracy=precision_and_recall(y_test, y_test_pred,threshold=0.1)
precision, recall, accuracy=precision_and_recall(y_test, y_test_pred,threshold=0.5)
precision, recall, accuracy=precision_and_recall(y_test, y_test_pred,threshold=0.9)

Threshold 0.1  --- Precision: 0.8458, Recall: 0.9955, Accuracy: 0.8861
Threshold 0.5  --- Precision: 0.9060, Recall: 0.9608, Accuracy: 0.9150
Threshold 0.9  --- Precision: 0.9621, Recall: 0.7437, Accuracy: 0.8251


### Use machine learning model

Calculate the probabilities for the tracks in those two submissions (small optimization: take also length of track into account, and after a couple of merged submissions, ask the probability of the track from the new submission to be at least C higher than the current probability; this latter option is not used in this kernel, but was used when merging >= 4 submissions). 

Then merge both submissions, based on the probabilities of its track candidates:

In [58]:
preds={}
preds[1]=get_predictions(resa1,hits,model)
preds[2]=get_predictions(resa2,hits,model)
print('Merge submission 0 and 1 into sub01:')
sub01=merge_with_probabilities(resa1,resa2,preds[1],preds[2],None,length_factor=0.5)
score = score_event_fast(truth, sub01)
print('Score:',score)

Merge submission 0 and 1 into sub01:
Score: 0.5651194312399999


## Expand tracks

In [60]:
mstd_vol={7:0,8:0,9:0,12:2,13:1,14:2,16:3,17:2,18:3}
mstd_size=[4,4,4,4,3,3,3,2,2,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
weights={'theta':0.1, 'phi':1}
nresa=expand_tracks(sub01,hits,5,16,5,7,mstd=8,dstd=0.00085,phik=3.3,max_dtheta=0.9*np.pi/2,mstd_vol=mstd_vol,mstd_size=mstd_size,weights=weights,nhipo=100)
nresa['event_id']=0
score = score_event_fast(truth, nresa)
print("Your score: ", score)

100%|██████████| 100/100 [00:07<00:00, 14.28it/s]


FloatProgress(value=0.0, description='calculating:', max=6763.0)

100%|██████████| 6763/6763 [02:29<00:00, 45.12it/s]


Your score:  0.6942199117400001
